# Pré-Requisitos

In [ ]:
# Instalando o Selenium
!pip install selenium

# Atualizando o Ubuntu para executar corretamento o apt-install
!apt-get update

# Instalando o ChromeDrive e Trazendo ele para a Pasta Local
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Utilizando o WebDriver do Selenium
from selenium import webdriver

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome('chromedriver',options=options)

# Importando as Bibliotecas Básicas
import pandas as pd
import time
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from selenium.webdriver.common.by import By

# Importando a Classe Cores
class bcolors:
    OKGREEN = '\033[92m' #GREEN
    OkBLUE = '\033[94m' #BLUE
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

# WebScraping do FlashScore

In [ ]:
jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odds_H':[],'Odds_D':[],'Odds_A':[]}

In [ ]:
def WebScraping():
  # Com o WebDrive a gente consegue a pedir a página (URL)
  wd_Chrome.get("https://www.flashscore.com/")

  # Pegando o ID dos Jogos
  id_jogos = []
  jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

  for i in jogos:
      id_jogos.append(i.get_attribute("id"))

  # Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
  id_jogos = [i[4:] for i in id_jogos]

  #jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odds_H':[],'Odds_D':[],'Odds_A':[]}

  for link in tqdm(id_jogos, total=len(id_jogos)):
      wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
      
      # Pegando as Informacoes Básicas do Jogo
      try:
          Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
          Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
          Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
          League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
          League = League.find_element(By.CSS_SELECTOR,'a').text
          Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
          Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
          Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
          Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
          
          # Match Odds
          wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
          time.sleep(2)
          celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
          
          Odds_H = 0
          Odds_D = 0
          Odds_A = 0
          
          if 'title="bet365"' in str(wd_Chrome.find_element(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')):
              for celula in celulas:
                  bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
                  bookie = bookie.get_attribute('title')
                  if ((bookie == 'bet365') & (Odds_H == 0)) | ((bookie == 'Betfair') & (Odds_H == 0)):
                      Odds_H = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                      Odds_D = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text 
                      Odds_A = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text
                  else:
                      pass
          else:
              for celula in celulas:
                  Odds_H = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                  Odds_D = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text 
                  Odds_A = celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text

      except:
          pass

      print(Date,Time,Country,League,Home,Away,Odds_H,Odds_D,Odds_A) 

      jogo['Date'].append(Date)
      jogo['Time'].append(Time)
      jogo['Country'].append(Country)
      jogo['League'].append(League)
      jogo['Home'].append(Home)
      jogo['Away'].append(Away)
      jogo['Odds_H'].append(Odds_H)
      jogo['Odds_D'].append(Odds_D)
      jogo['Odds_A'].append(Odds_A)

# Iniciando a Raspagem de Dados

In [ ]:
WebScraping()

In [ ]:
df_today = pd.DataFrame(jogo)
df_today = df_today[(df_today.Odds_H != 0)]
df_today['Time'] = pd.to_datetime(df_today['Time']) - pd.DateOffset(hours=3)
df_today = df_today.sort_values(['Time'])
df_today.reset_index(inplace=True, drop=True)
df_today.index = df_today.index.set_names(['Nº'])
df_today = df_today.rename(index=lambda x: x + 1)
df_today.columns = ['DateTime', 'Date', 'Country','League','Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A']
df_today = df_today[['Date', 'Country','League','Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A']]
df_today

# Machine Learning

In [ ]:
# Importando os DataFrames
df_ENG1 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/England%20Premier%20League_20212022.xlsx?raw=true')
df_ENG2 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/England%20Championship_20212022.xlsx?raw=true')
df_FRA1 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/France%20Ligue%201_20212022.xlsx?raw=true')
df_FRA2 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/France%20Ligue%202_20212022.xlsx?raw=true')
df_GER1 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Germany%20Bundesliga_20212022.xlsx?raw=true')
df_GER2 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Germany%202.%20Bundesliga_20212022.xlsx?raw=true')
df_ITA1 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Italy%20Serie%20A_20212022.xlsx?raw=true')
df_ITA2 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Italy%20Serie%20B_20212022.xlsx?raw=true')
df_SPA1 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Spain%20La%20Liga_20212022.xlsx?raw=true')
df_SPA2 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Spain%20Segunda%20Divisi%C3%B3n_20212022.xlsx?raw=true')
# Concatenando os DataFrames
df = [df_ENG1,df_ENG2,df_FRA1,df_FRA2,df_GER1,df_GER2,df_ITA1,df_ITA1,df_SPA1,df_SPA2]
df = pd.concat(df)
df = df[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]
# Criando a Coluna de 0 e 1 para análise do Modelo de Machine Learning
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'BackHome'] = 1
df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'BackHome'] = 0
#Separando os DataFrames
df1 = df[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','BackHome']]
df1 = df1.dropna()
df2 = df[['FT_Odds_H','FT_Odds_D','FT_Odds_A','BackHome']]
df2 = df2.dropna()
# convertendo os arrays (matriz)
df1_x = df1.iloc[:,:].values
df2_x = df2.iloc[:,:-1].values
df2_y = df2.iloc[:,-1].values
# separando os dados para treino e para teste
x_dados_treinamento, x_dados_teste = train_test_split(df1_x, test_size=0.20, random_state = 0)
df1 = pd.DataFrame(x_dados_teste)
df1.columns = ['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','BackHome']
# separando os dados para treino e para teste
x_dados_treinamento, x_dados_teste, y_dados_treinamento, y_dados_teste = train_test_split(df2_x, df2_y, test_size=0.20, random_state = 0)
# Definições de Parâmetros e dados de entrada e resultado para treinamento
knn_dados = KNeighborsClassifier(n_neighbors=3, metric="minkowski", p=2)
knn_dados.fit(x_dados_treinamento, y_dados_treinamento)
# Previsões
previsoes_knn = knn_dados.predict(x_dados_teste)
df1["Previsões"] = previsoes_knn
stake = 1
win_Back = stake * (df1.FT_Odds_H - 1)
lose_Back = -stake

df1.loc[(df1['Previsões'] == 1) & (df1['BackHome'] == 1), 'Profit'] = win_Back
df1.loc[(df1['Previsões'] == 1) & (df1['BackHome'] == 0), 'Profit'] = lose_Back
df1.loc[(df1['Previsões'] == 0) & (df1['BackHome'] == 1), 'Profit'] = 0
df1.loc[(df1['Previsões'] == 0) & (df1['BackHome'] == 0), 'Profit'] = 0

filtro = df1.Previsões == 1
df0 = df1[filtro]

# Ajustando o Índice
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)

In [ ]:
def MachineLearningKNN():

    for a,b,c,d,e,f in zip(df_today.Date,df_today.Home,df_today.Away,df_today.FT_Odds_H,df_today.FT_Odds_D,df_today.FT_Odds_A):
    
        Date = a
        Home = b
        Away = c
        FT_Odds_H = d
        FT_Odds_D = e
        FT_Odds_A = f
        
        Odds = [FT_Odds_H,FT_Odds_D,FT_Odds_A]

        # Transformando a Lista em uma Matriz
        odds_jogo = np.array(Odds)

        previsao_jogo_knn = odds_jogo.reshape(1, -1)
        previsao_jogo_knn = knn_dados.predict(previsao_jogo_knn)
        
        
        print('');print("Jogo:",Home,'x',Away);print("Horário:",Date);print("Odd H:", FT_Odds_H)

        if previsao_jogo_knn == 1:
          print("Opinião do KNN:" + bcolors.OKGREEN + "Fazer Back Mandante" + bcolors.RESET)
        else:
          print("Opinião do KNN:" + bcolors.FAIL + "Ficar de Fora desse jogo" + bcolors.RESET)



# Teste - Machine Learning

In [ ]:
df0['Profit_acu'] = df0.Profit.cumsum()
df0.Profit_acu.plot()
df0.Profit_acu.tail(1)

# Aplicação - Machine Learning

In [ ]:
MachineLearningKNN()